# Credit Risk Analysis — SQL & Data Modeling (MySQL)

Neste notebook, realizo a modelagem relacional e análises em SQL
sobre uma base de dados de crédito, simulando um ambiente real
de uma fintech.

O objetivo é gerar indicadores estratégicos de inadimplência
que possam apoiar decisões de concessão de crédito e gestão de risco.


In [3]:
import pandas as pd
import mysql.connector

df = pd.read_csv(
    "C:/Users/Eliane/Downloads/Arquivos Facul/credit-risk-analysis/data/raw/credit_data.csv",
    sep=";",
    header=1
)

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Limites1007@",
    database="credit_risk"
)

cursor = conn.cursor()



In [4]:
cursor.execute("""
DROP TABLE IF EXISTS resultado_default;
""")

cursor.execute("""
DROP TABLE IF EXISTS pagamentos;
""")

cursor.execute("""
DROP TABLE IF EXISTS clientes;
""")

cursor.execute("""
CREATE TABLE clientes (
    id INT PRIMARY KEY,
    limit_bal INT,
    sex INT,
    education INT,
    marriage INT,
    age INT
);
""")

cursor.execute("""
CREATE TABLE pagamentos (
    id INT,
    pay_0 INT,
    pay_2 INT,
    pay_3 INT,
    pay_4 INT,
    pay_5 INT,
    pay_6 INT,
    FOREIGN KEY (id) REFERENCES clientes(id)
);
""")

cursor.execute("""
CREATE TABLE resultado_default (
    id INT,
    default_next_month INT,
    FOREIGN KEY (id) REFERENCES clientes(id)
);
""")


In [6]:
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO clientes VALUES (%s,%s,%s,%s,%s,%s)
    """, (
        int(row['ID']),
        int(row['LIMIT_BAL']),
        int(row['SEX']),
        int(row['EDUCATION']),
        int(row['MARRIAGE']),
        int(row['AGE'])
    ))

conn.commit()


IntegrityError: 1062 (23000): Duplicate entry '1' for key 'clientes.PRIMARY'

In [7]:
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO pagamentos VALUES (%s,%s,%s,%s,%s,%s,%s)
    """, (
        int(row['ID']),
        int(row['PAY_0']),
        int(row['PAY_2']),
        int(row['PAY_3']),
        int(row['PAY_4']),
        int(row['PAY_5']),
        int(row['PAY_6'])
    ))

conn.commit()


In [8]:
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO resultado_default VALUES (%s,%s)
    """, (
        int(row['ID']),
        int(row['default payment next month'])
    ))

conn.commit()


In [9]:
cursor.execute("SHOW TABLES")
cursor.fetchall()


[('clientes',), ('pagamentos',), ('resultado_default',)]

In [10]:
pd.read_sql("SELECT COUNT(*) FROM clientes", conn)


C:\Users\Eliane\AppData\Local\Temp\ipykernel_1228\293614245.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT COUNT(*) FROM clientes", conn)


,COUNT(*)
0,30000


In [11]:
query = """
SELECT 
    default_next_month AS inadimplente,
    COUNT(*) AS total,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM resultado_default), 2) AS percentual
FROM resultado_default
GROUP BY default_next_month;
"""
pd.read_sql(query, conn)


C:\Users\Eliane\AppData\Local\Temp\ipykernel_1228\3819018777.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,inadimplente,total,percentual
0,1,6636,22.12
1,0,23364,77.88


In [12]:
query = """
SELECT 
    r.default_next_month AS inadimplente,
    ROUND(AVG(c.limit_bal),2) AS limite_medio
FROM clientes c
JOIN resultado_default r ON c.id = r.id
GROUP BY r.default_next_month;
"""
pd.read_sql(query, conn)


C:\Users\Eliane\AppData\Local\Temp\ipykernel_1228\1674586397.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,inadimplente,limite_medio
0,1,130109.66
1,0,178099.73


In [13]:
query = """
SELECT 
    CASE
        WHEN age < 30 THEN 'Até 29'
        WHEN age BETWEEN 30 AND 39 THEN '30–39'
        WHEN age BETWEEN 40 AND 49 THEN '40–49'
        ELSE '50+'
    END AS faixa_etaria,
    ROUND(AVG(r.default_next_month) * 100,2) AS taxa_inadimplencia
FROM clientes c
JOIN resultado_default r ON c.id = r.id
GROUP BY faixa_etaria
ORDER BY faixa_etaria;
"""
pd.read_sql(query, conn)


C:\Users\Eliane\AppData\Local\Temp\ipykernel_1228\627810640.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,faixa_etaria,taxa_inadimplencia
0,30–39,20.25
1,40–49,22.97
2,50+,25.30
3,Até 29,22.84


In [14]:
query = """
SELECT 
    c.id,
    c.limit_bal,
    c.age,
    c.education,
    p.pay_0,
    r.default_next_month
FROM clientes c
JOIN pagamentos p ON c.id = p.id
JOIN resultado_default r ON c.id = r.id;
"""
df_powerbi = pd.read_sql(query, conn)
df_powerbi.head()


C:\Users\Eliane\AppData\Local\Temp\ipykernel_1228\2091567639.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_powerbi = pd.read_sql(query, conn)


,id,limit_bal,age,education,pay_0,default_next_month
0,1,20000,24,2,2,1
1,2,120000,26,2,-1,1
2,3,90000,34,2,0,0
3,4,50000,37,2,0,0
4,5,50000,57,2,-1,0
